In [1]:
import pandas as pd
from requests_html import HTMLSession
from tqdm import tqdm
import numpy as np
from tqdm import tqdm

In [2]:
links = []
for x in range(0,14):
    links.append(f'https://www.lamudi.com.co/quindio/for-sale/?page={x}')

links.append('https://www.lamudi.com.co/la-guajira/for-sale/')
links.append('https://www.lamudi.com.co/amazonas/for-sale/')
links.append('https://www.lamudi.com.co/choco/for-sale/')

In [3]:
links_pages = []

In [4]:

for r in tqdm(links):
    s = HTMLSession()
    r = s.get(r)
    urls = r.html.find('div.clp-wrapper > div:nth-child(2) > div > div > div.ListingCell-AllInfo.ListingUnit > a')
    for i in urls:
        links_pages.append(i.attrs['href'])

100%|██████████| 17/17 [00:16<00:00,  1.01it/s]


In [5]:
len(links_pages)

432

In [6]:
data = []

In [7]:
for i in tqdm(links_pages):
    s = HTMLSession()
    r = s.get(i)
    txt = r.html.find('script:contains("dataLayer")',first = True)
    try:
        precio = txt.search('"price_formatted": "{precio}"')['precio'].strip('$')
        precio = pd.to_numeric(precio.strip(' ').replace(',',''))
    except:
        precio = np.nan
    try:
        baños = txt.search('"bathrooms":{baños},')['baños'].strip('\n                ')
    except:
        baños = np.nan
    try:
        habitaciones = txt.search('"bedrooms":{habitaciones},')['habitaciones'].strip('\n                ')
    except:
        habitaciones = np.nan
    try:
        area_total = txt.search('"building_size":{area_total},')['area_total'].strip('\n                ')
    except:
        area_total = np.nan
    try:
        año_construido = txt.search('"year_built":{año_construido},')['año_construido'].strip('\n                ')
    except:
        año_construido = np.nan
    try:
        antigüedad = 2022- pd.to_numeric(año_construido)
    except:
        antigüedad = np.nan
    try:
        nombre_inmobiliaria = txt.search('"agency_name": "{nombre_inmobiliaria}"')['nombre_inmobiliaria']
    except:
        nombre_inmobiliaria = np.nan
    try:
        tipo_inmueble = txt.search('"attribute_set_name": "{tipo_inmueble}"')['tipo_inmueble']
    except:
        tipo_inmueble = np.nan
    try:
        fecha_publicacion = txt.search('"listing_start": "{fecha_publicacion}"')['fecha_publicacion']
    except:
        fecha_publicacion = np.nan
    try:
        titulo = txt.search('"title": "{titulo}",')['titulo']
    except:
        titulo = np.nan
    try:
        latitud = txt.search('"location_latitude": "{latitud}"')['latitud']
    except:
        latitud = np.nan
    try:
        longitud = txt.search('"location_latitude": "{longitud}"')['longitud']
    except:
        longitud = np.nan
    try:
        departamento = txt.search('"listing_region": "{departamento}"')['departamento']
    except:
        departamento = np.nan
    try:
        municipio = txt.search('"listing_city": "{municipio}"')['municipio']
    except:
        municipio = np.nan
    try:
        descripcion = txt.search(' "text": "{descripcion}"')['descripcion']
    except:
        descripcion = np.nan
    try:
        estacionamientos = txt.search('"car_spaces":{estacionamientos},')['estacionamientos'].strip('\n                ')
    except:
        estacionamientos = np.nan
    datos = {'precio' : precio, 'baños' : baños,'habitaciones':habitaciones,'area_total':area_total, 'año_construido':año_construido,'antigüedad':antigüedad,'nombre_inmobiliaria':nombre_inmobiliaria,
    'tipo_inmueble' : tipo_inmueble,'fecha_publicacion' : fecha_publicacion,'titulo' : titulo, 'latitud' : latitud, 'longitud' : longitud,'departamento' : departamento,'municipio' : municipio,
    'descripcion' : descripcion, 'estacionamientos' : estacionamientos, 'url_page':i}
    data.append(datos)



100%|██████████| 432/432 [17:11<00:00,  2.39s/it]


In [8]:
df = pd.DataFrame(data)

In [9]:
df.shape

(432, 17)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.to_csv('lamudi_datos_20220926.csv',sep = ';',index=False, encoding = 'utf-8')